In [1]:
import json
import requests
from datetime import datetime
import pandas as pd
#import numpy as np

In [2]:
def make_and_save_api_call(endpoint, params, filename):
    call = requests.get(endpoint.format(**params))
    response = call.json()
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(response, file)

In [3]:
games_endpoint = "https://api.collegefootballdata.com/games?year={year}"

In [4]:
make_and_save_api_call(games_endpoint, {"year": "2012"}, "raw_data/games_endpoint_2012.json")
make_and_save_api_call(games_endpoint, {"year": "2013"}, "raw_data/games_endpoint_2013.json")
make_and_save_api_call(games_endpoint, {"year": "2014"}, "raw_data/games_endpoint_2014.json")
make_and_save_api_call(games_endpoint, {"year": "2015"}, "raw_data/games_endpoint_2015.json")
make_and_save_api_call(games_endpoint, {"year": "2016"}, "raw_data/games_endpoint_2016.json")
make_and_save_api_call(games_endpoint, {"year": "2017"}, "raw_data/games_endpoint_2017.json")
make_and_save_api_call(games_endpoint, {"year": "2018"}, "raw_data/games_endpoint_2018.json")

game data does not store attendance data before 2012

In [5]:
games_2012 = pd.read_json("raw_data/games_endpoint_2012.json")
games_2013 = pd.read_json("raw_data/games_endpoint_2013.json")
games_2014 = pd.read_json("raw_data/games_endpoint_2014.json")
games_2015 = pd.read_json("raw_data/games_endpoint_2015.json")
games_2016 = pd.read_json("raw_data/games_endpoint_2016.json")
games_2017 = pd.read_json("raw_data/games_endpoint_2017.json")
games_2018 = pd.read_json("raw_data/games_endpoint_2018.json")

In [6]:
all_games = pd.concat([games_2012, games_2013, games_2014, games_2015, games_2016, games_2017, games_2018], ignore_index=True)
all_games.head()

,attendance,away_conference,away_line_scores,away_points,away_team,conference_game,home_conference,home_line_scores,home_points,home_team,id,neutral_site,season,season_type,start_date,venue,venue_id,week
0,38393.0,SEC,"[7, 3, 0, 7]",17,South Carolina,True,SEC,"[0, 10, 3, 0]",13,Vanderbilt,322430238,False,2012,regular,2012-08-30T19:00:00.000Z,Vanderbilt Stadium,3973.0,1
1,12616.0,Conference USA,"[7, 28, 14, 7]",56,UCF,False,Mid-American,"[0, 0, 7, 7]",14,Akron,322432006,False,2012,regular,2012-08-30T19:00:00.000Z,Summa Field at InfoCision Stadium,3768.0,1
2,15250.0,None,"[10, 14, 3, 0]",27,Southeast Missouri State,False,Mid-American,"[10, 14, 7, 7]",38,Central Michigan,322432117,False,2012,regular,2012-08-30T19:00:00.000Z,Kelly/Shorts Stadium,3786.0,1
3,12725.0,Mid-American,"[0, 13, 0, 13]",26,Eastern Michigan,True,Mid-American,"[10, 3, 21, 3]",37,Ball State,322432050,False,2012,regular,2012-08-30T19:00:00.000Z,Scheumann Stadium,3919.0,1
4,15121.0,None,"[0, 7, 0, 14]",21,Towson,False,Mid-American,"[17, 10, 7, 7]",41,Kent State,322432309,False,2012,regular,2012-08-30T19:00:00.000Z,Dix Stadium,3696.0,1


In [7]:
len(all_games[all_games["attendance"] == 0])

911

In [8]:
all_games = all_games[all_games["attendance"] > 0]

In [9]:
venues_endpoint = "https://api.collegefootballdata.com/venues"
make_and_save_api_call(venues_endpoint, {}, "raw_data/venues_endpoint.json")
venues = pd.read_json("raw_data/venues_endpoint.json")
venues.head()

,capacity,city,country_code,dome,elevation,grass,id,location,name,state,year_constructed,zip
0,10000,Tuskegee,US,0.0,122.800000,NaN,4747,None,Abbott Memorial Alumni Stadium,AL,1925.0,NaN
1,28500,Baton Rouge,US,0.0,20.242132,NaN,3599,"{'x': 30.5221461, 'y': -91.1896087}",Ace W. Mumford Stadium,LA,1928.0,NaN
2,30343,Las Cruces,US,0.0,1208.201294,0.0,3601,"{'x': 32.2796202, 'y': -106.7411148}",Aggie Memorial Stadium,NM,1978.0,88003.0
3,21500,Greensboro,US,0.0,235.122452,NaN,3603,"{'x': 36.0814337, 'y': -79.7700391}",Aggie Stadium,NC,1981.0,NaN
4,10849,Davis,US,0.0,11.382592,NaN,3602,"{'x': 38.5365266, 'y': -121.7627936}",Aggie Stadium,CA,2007.0,NaN


join on venue id:

In [10]:
venues = venues.rename(columns={"id": "venue_id"})


In [11]:
attendance_data = pd.merge(all_games, venues, on="venue_id")

In [12]:
attendance_data.head()

,attendance,away_conference,away_line_scores,away_points,away_team,conference_game,home_conference,home_line_scores,home_points,home_team,...,city,country_code,dome,elevation,grass,location,name,state,year_constructed,zip
0,38393.0,SEC,"[7, 3, 0, 7]",17,South Carolina,True,SEC,"[0, 10, 3, 0]",13,Vanderbilt,...,Nashville,US,0.0,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212.0
1,35491.0,None,"[0, 0, 0, 0]",0,Presbyterian College,False,SEC,"[10, 17, 28, 3]",58,Vanderbilt,...,Nashville,US,0.0,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212.0
2,40350.0,SEC,"[0, 11, 7, 13]",31,Florida,True,SEC,"[7, 0, 0, 10]",17,Vanderbilt,...,Nashville,US,0.0,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212.0
3,40350.0,SEC,"[0, 10, 0, 3]",13,Auburn,True,SEC,"[7, 3, 7, 0]",17,Vanderbilt,...,Nashville,US,0.0,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212.0
4,32227.0,Mid-American,"[0, 0, 0, 7]",7,UMass,False,SEC,"[7, 14, 28, 0]",49,Vanderbilt,...,Nashville,US,0.0,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212.0


In [13]:
attendance_data["attendance_percentage"] = attendance_data["attendance"] / attendance_data["capacity"]

In [14]:
#https://stackoverflow.com/questions/21287624/convert-pandas-column-containing-nans-to-dtype-int
attendance_data["zip"] = attendance_data["zip"].astype('Int64')
attendance_data["start_date"] = pd.to_datetime(attendance_data["start_date"]).dt.date

In [15]:
rankings_endpoint = "https://api.collegefootballdata.com/rankings?year={year}"
make_and_save_api_call(rankings_endpoint, {"year": "2012"}, "raw_data/rankings_endpoint_2012.json")
make_and_save_api_call(rankings_endpoint, {"year": "2013"}, "raw_data/rankings_endpoint_2013.json")
make_and_save_api_call(rankings_endpoint, {"year": "2014"}, "raw_data/rankings_endpoint_2014.json")
make_and_save_api_call(rankings_endpoint, {"year": "2015"}, "raw_data/rankings_endpoint_2015.json")
make_and_save_api_call(rankings_endpoint, {"year": "2016"}, "raw_data/rankings_endpoint_2016.json")
make_and_save_api_call(rankings_endpoint, {"year": "2017"}, "raw_data/rankings_endpoint_2017.json")
make_and_save_api_call(rankings_endpoint, {"year": "2018"}, "raw_data/rankings_endpoint_2018.json")

In [16]:
rankings_2012 = pd.read_json("raw_data/rankings_endpoint_2012.json")
rankings_2013 = pd.read_json("raw_data/rankings_endpoint_2013.json")
rankings_2014 = pd.read_json("raw_data/rankings_endpoint_2014.json")
rankings_2015 = pd.read_json("raw_data/rankings_endpoint_2015.json")
rankings_2016 = pd.read_json("raw_data/rankings_endpoint_2016.json")
rankings_2017 = pd.read_json("raw_data/rankings_endpoint_2017.json")
rankings_2018 = pd.read_json("raw_data/rankings_endpoint_2018.json")

In [17]:
datasets = [rankings_2012, rankings_2013, rankings_2014, rankings_2015, rankings_2016, rankings_2017, rankings_2018]

ap_polls = []
years = []
weeks = []

for dataset in datasets:
    for index, row in dataset.iterrows():
        weeks.append(row.week)
        years.append(row.season)
        polls = row.polls
        for i in range(len(polls)):
            # get poll names
            if polls[i]["poll"] == "AP Top 25":
                ranked_teams = polls[i]["ranks"]
                schools = []
                for ranking in ranked_teams:
                    schools.append(ranking["school"])
                ap_polls.append(schools)

In [18]:
ap_ranking_data = pd.DataFrame()
ap_ranking_data["ap_poll"] = ap_polls 
ap_ranking_data["year"] = years
ap_ranking_data["week"] = weeks
ap_ranking_data.head()

,ap_poll,year,week
0,"[Utah State, Ohio State, Michigan, Nebraska, N...",2012,14
1,"[Rutgers, Louisville, Utah State, Ohio State, ...",2012,13
2,"[Rutgers, Louisiana Tech, Louisville, Ohio Sta...",2012,12
3,"[Rutgers, Louisiana Tech, Louisville, Ohio Sta...",2012,11
4,"[Louisiana Tech, Louisville, Ohio State, Nebra...",2012,10


In [19]:
home_team_ranking_indicator = []
away_team_ranking_indicator = []
# make ranking indicator

for index, row in attendance_data.iterrows():
    week_rankings = ap_ranking_data[(ap_ranking_data["year"] == row.season) & (ap_ranking_data["week"] == row.week)]
    if row.home_team in week_rankings.ap_poll.iloc[0]:
        home_team_ranking_indicator.append(1)
    else:
        home_team_ranking_indicator.append(0)
        
    if row.away_team in week_rankings.ap_poll.iloc[0]:
        away_team_ranking_indicator.append(1)
    else:
        away_team_ranking_indicator.append(0)

In [20]:
attendance_data["home_team_ranked_ind"] = home_team_ranking_indicator
attendance_data["away_team_ranked_ind"] = away_team_ranking_indicator
attendance_data.head()

,attendance,away_conference,away_line_scores,away_points,away_team,conference_game,home_conference,home_line_scores,home_points,home_team,...,elevation,grass,location,name,state,year_constructed,zip,attendance_percentage,home_team_ranked_ind,away_team_ranked_ind
0,38393.0,SEC,"[7, 3, 0, 7]",17,South Carolina,True,SEC,"[0, 10, 3, 0]",13,Vanderbilt,...,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212,0.951499,0,1
1,35491.0,None,"[0, 0, 0, 0]",0,Presbyterian College,False,SEC,"[10, 17, 28, 3]",58,Vanderbilt,...,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212,0.879579,0,0
2,40350.0,SEC,"[0, 11, 7, 13]",31,Florida,True,SEC,"[7, 0, 0, 10]",17,Vanderbilt,...,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212,1.000000,0,1
3,40350.0,SEC,"[0, 10, 0, 3]",13,Auburn,True,SEC,"[7, 3, 7, 0]",17,Vanderbilt,...,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212,1.000000,0,0
4,32227.0,Mid-American,"[0, 0, 0, 7]",7,UMass,False,SEC,"[7, 14, 28, 0]",49,Vanderbilt,...,159.919464,0.0,"{'x': 36.1440455, 'y': -86.8088942}",Vanderbilt Stadium,TN,1922.0,37212,0.798686,0,0


In [21]:
def get_noaa_data(url, data_type, header):
    r = requests.get(url, data_type, headers=header)
    return r.content

In [22]:
weather_responses = []

token = "put_token_here"
creds = dict(token=token)
dtype = "dataset"

for index, row in attendance_data.iterrows():
    endpoint = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:%s&startdate=%s&enddate=%s" % (row.zip, row.start_date, row.start_date)
    weather_responses.append(get_noaa_data(endpoint, dtype, creds))

ConnectionError: HTTPSConnectionPool(host='www.ncdc.noaa.gov', port=443): Max retries exceeded with url: /cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:37996&startdate=2017-11-18&enddate=2017-11-18&dataset (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000026456D11588>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [51]:
rain = []
temp_max = []
temp_min = []

for response in weather_responses:
    response_json = json.loads(response)
    if response_json:
        for i in range(len(response_json["results"])):
            if response_json["results"][i]["datatype"] == "PRCP":
                rain.append(response_json["results"][i]["value"])
            elif response_json["results"][i]["datatype"] == "TMAX":
                temp_max.append(response_json["results"][i]["value"])
            elif response_json["results"][i]["datatype"] == "TMIN":
                temp_min.append(response_json["results"][i]["value"])
    else:
        rain.append(None)
        temp_max.append(None)
        temp_min.append(None)

In [56]:
attendance_data.shape[0]

4833

In [57]:
rain = rain + [None] * (attendance_data.shape[0] - len(rain))
temp_max = temp_max + [None] * (attendance_data.shape[0] - len(temp_max))
temp_min = temp_min + [None] * (attendance_data.shape[0] - len(temp_min))

In [58]:
attendance_data["rain"] = rain
attendance_data["max_temp"] = temp_max
attendance_data["min_temp"] = temp_min

In [ ]:
#weather_endpoint = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=TMAX,PRCP,TMIN&stations={stations}&startDate={startDate}&endDate={endDate}&includeAttributes=true&format=json"

In [ ]:
#make_and_save_api_call(weather_endpoint, {"stations": "USC00456096", "startDate": "2007-01-01", "endDate": "2007-01-02"}, "weather_test.json")

In [ ]:
#weather_test = pd.read_json("weather_test.json")
#weather_test

In [ ]:
#weather_endpoint = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?lat={lat}&lon={lon}&product=time-series&begin=2004-01-01T00:00:00&end=2019-04-20T00:00:00&maxt=maxt&mint=mint"

In [ ]:
#for i, row in game_and_venue_data.iterrows():
#    if row["location"]:
#        lat = row["location"]["x"]
#        lon = row["location"]["y"]
#    zip_code = row["zip"]
#    kickoff = row["start_date"]

In [59]:
attendance_data.to_csv("processed_data/seasons_2012-2018_attendance_data.csv")